In [1]:
import pandas as pd 

In [2]:
#DJF
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
DJFReports = []
DJFDeath = []
DJFInjury = []
DJFCost = []
DJFReportDaysList = []
DJFTornadoWidth = []
DJFTornadoTrack = []
DJFHailMax = []
DJFWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date1 = pd.to_datetime('12-01-1980')
Date2 = pd.to_datetime('02-28-1981')
DJFCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while DJFCount <= 29:
    x = df.loc[Date1:Date2]
    maxes = x.groupby(['EVENT_TYPE']).max()
    DJFReports.insert(DJFCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    DJFDeath.insert(DJFCount,y)
    z = x.INJURIES_DIRECT.sum()
    DJFInjury.insert(DJFCount,z)
    a = x.TOR_LENGTH.sum()
    DJFTornadoTrack.insert(DJFCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        DJFTornadoWidth.insert(DJFCount,b)
    else: 
        DJFTornadoWidth.insert(DJFCount,0)
    c = x.Total_Damages.sum()
    DJFCost.insert(DJFCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            DJFHailMax.insert(DJFCount,d)
        else: 
            DJFHailMax.insert(DJFCount,0)
    except KeyError: 
        DJFHailMax.insert(DJFCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            DJFWindMax.insert(DJFCount,d)
        else: 
            DJFWindMax.insert(DJFCount,0)
    except KeyError:
        DJFWindMax.insert(DJFCount,0)
    
    DJFReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        DJFReportDaysList.insert(DJFCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                DJFReportDays = DJFReportDays + 1
        DJFReportDaysList.insert(DJFCount, DJFReportDays)
     
    #if the year was a leap year, then this sets the end date back to Feb. 28
    if Date2.is_leap_year:
        Date2 = Date2 - pd.DateOffset(days=1)
    #adds a year to the end date range for the next cycle of the while statement
    Date1 = Date1 + pd.DateOffset(years=1)
    Date2 = Date2 + pd.DateOffset(years=1)
    #if the upcoming year is a leap year, add 1 day to account for Feb. 29
    if Date2.is_leap_year:
        Date2 = Date2 + pd.DateOffset(days=1)
    DJFCount = DJFCount + 1
    
#Create dataframes for each list
DJFdf1 = pd.DataFrame({'DJF Reports': DJFReports})
DJFdf2 = pd.DataFrame({'DJF Report Days': DJFReportDaysList})
DJFdf3 = pd.DataFrame({'DJF Fatalities': DJFDeath})
DJFdf4 = pd.DataFrame({'DJF Injuries': DJFInjury})
DJFdf5 = pd.DataFrame({'DJF Tornado Track': DJFTornadoTrack})
DJFdf6 = pd.DataFrame({'DJF Damages': DJFCost})
DJFdf7 = pd.DataFrame({'DJF Largest Hail': DJFHailMax})
DJFdf8 = pd.DataFrame({'DJF Strongest Wind': DJFWindMax})
DJFdf9 = pd.DataFrame({'DJF Widest Tornado': DJFTornadoWidth})

#concatenate the dataframes together
DJFdftot = pd.concat([
    DJFdf1, DJFdf2, DJFdf3, DJFdf4, 
    DJFdf5, DJFdf6, DJFdf7, DJFdf8, DJFdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
DJFdftot = DJFdftot.set_index([years])

In [3]:
#JFM
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
JFMReports = []
JFMDeath = []
JFMInjury = []
JFMCost = []
JFMReportDaysList = []
JFMTornadoWidth = []
JFMTornadoTrack = []
JFMHailMax = []
JFMWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date3 = pd.to_datetime('01-01-1981')
Date4 = pd.to_datetime('03-31-1981')
JFMCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while JFMCount <= 29:
    x = df.loc[Date3:Date4]
    maxes = x.groupby(['EVENT_TYPE']).max()
    JFMReports.insert(JFMCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    JFMDeath.insert(JFMCount,y)
    z = x.INJURIES_DIRECT.sum()
    JFMInjury.insert(JFMCount,z)
    a = x.TOR_LENGTH.sum()
    JFMTornadoTrack.insert(JFMCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        JFMTornadoWidth.insert(JFMCount,b)
    else: 
        JFMTornadoWidth.insert(JFMCount,0)
    c = x.Total_Damages.sum()
    JFMCost.insert(DJFCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            JFMHailMax.insert(JFMCount,d)
        else: 
            JFMHailMax.insert(JFMCount,0)
    except KeyError: 
        JFMHailMax.insert(JFMCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            JFMWindMax.insert(JFMCount,d)
        else: 
            JFMWindMax.insert(JFMCount,0)
    except KeyError:
        JFMWindMax.insert(JFMCount,0)
    
    JFMReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        JFMReportDaysList.insert(JFMCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                JFMReportDays = JFMReportDays + 1
        JFMReportDaysList.insert(JFMCount, JFMReportDays)
    Date3 = Date3 + pd.DateOffset(years=1)
    Date4 = Date4 + pd.DateOffset(years=1)
    JFMCount = JFMCount + 1
    
#Create dataframes for each list
JFMdf1 = pd.DataFrame({'JFM Reports': JFMReports})
JFMdf2 = pd.DataFrame({'JFM Report Days': JFMReportDaysList})
JFMdf3 = pd.DataFrame({'JFM Fatalities': JFMDeath})
JFMdf4 = pd.DataFrame({'JFM Injuries': JFMInjury})
JFMdf5 = pd.DataFrame({'JFM Tornado Track': JFMTornadoTrack})
JFMdf6 = pd.DataFrame({'JFM Damages': JFMCost})
JFMdf7 = pd.DataFrame({'JFM Largest Hail': JFMHailMax})
JFMdf8 = pd.DataFrame({'JFM Strongest Wind': JFMWindMax})
JFMdf9 = pd.DataFrame({'JFM Widest Tornado': JFMTornadoWidth})

#concatenate the dataframes together
JFMdftot = pd.concat([
    JFMdf1, JFMdf2, JFMdf3, JFMdf4,
    JFMdf5, JFMdf6, JFMdf7, JFMdf8, JFMdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
JFMdftot = JFMdftot.set_index([years])

In [4]:
#FMA
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
FMAReports = []
FMADeath = []
FMAInjury = []
FMACost = []
FMAReportDaysList = []
FMATornadoWidth = []
FMATornadoTrack = []
FMAHailMax = []
FMAWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date5 = pd.to_datetime('02-01-1981')
Date6 = pd.to_datetime('04-30-1981')
FMACount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while FMACount <= 29:
    x = df.loc[Date5:Date6]
    maxes = x.groupby(['EVENT_TYPE']).max()
    FMAReports.insert(FMACount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    FMADeath.insert(FMACount,y)
    z = x.INJURIES_DIRECT.sum()
    FMAInjury.insert(FMACount,z)
    a = x.TOR_LENGTH.sum()
    FMATornadoTrack.insert(FMACount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        FMATornadoWidth.insert(FMACount,b)
    else: 
        FMATornadoWidth.insert(FMACount,0)
    c = x.Total_Damages.sum()
    FMACost.insert(FMACount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            FMAHailMax.insert(FMACount,d)
        else: 
            FMAHailMax.insert(FMACount,0)
    except KeyError: 
        FMAHailMax.insert(FMACount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            FMAWindMax.insert(FMACount,d)
        else: 
            FMAWindMax.insert(FMACount,0)
    except KeyError:
        FMAWindMax.insert(FMACount,0)
    
    FMAReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        FMAReportDaysList.insert(FMACount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                FMAReportDays = FMAReportDays + 1
        FMAReportDaysList.insert(FMACount, FMAReportDays)
    Date5 = Date5 + pd.DateOffset(years=1)
    Date6 = Date6 + pd.DateOffset(years=1)
    FMACount = FMACount + 1
    
#Create dataframes for each list
FMAdf1 = pd.DataFrame({'FMA Reports': FMAReports})
FMAdf2 = pd.DataFrame({'FMA Report Days': FMAReportDaysList})
FMAdf3 = pd.DataFrame({'FMA Fatalities': FMADeath})
FMAdf4 = pd.DataFrame({'FMA Injuries': FMAInjury})
FMAdf5 = pd.DataFrame({'FMA Tornado Track': FMATornadoTrack})
FMAdf6 = pd.DataFrame({'FMA Damages': FMACost})
FMAdf7 = pd.DataFrame({'FMA Largest Hail': FMAHailMax})
FMAdf8 = pd.DataFrame({'FMA Strongest Wind': FMAWindMax})
FMAdf9 = pd.DataFrame({'FMA Widest Tornado': FMATornadoWidth})

#concatenate the dataframes together
FMAdftot = pd.concat([
    FMAdf1, FMAdf2, FMAdf3, FMAdf4, 
    FMAdf5, FMAdf6, FMAdf7, FMAdf8, FMAdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
FMAdftot = FMAdftot.set_index([years])

In [5]:
#MAM
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
MAMReports = []
MAMDeath = []
MAMInjury = []
MAMCost = []
MAMReportDaysList = []
MAMTornadoWidth = []
MAMTornadoTrack = []
MAMHailMax = []
MAMWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date7 = pd.to_datetime('03-01-1981')
Date8 = pd.to_datetime('05-31-1981')
MAMCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while MAMCount <= 29:
    x = df.loc[Date7:Date8]
    maxes = x.groupby(['EVENT_TYPE']).max()
    MAMReports.insert(MAMCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    MAMDeath.insert(MAMCount,y)
    z = x.INJURIES_DIRECT.sum()
    MAMInjury.insert(MAMCount,z)
    a = x.TOR_LENGTH.sum()
    MAMTornadoTrack.insert(MAMCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        MAMTornadoWidth.insert(MAMCount,b)
    else: 
        MAMTornadoWidth.insert(MAMCount,0)
    c = x.Total_Damages.sum()
    MAMCost.insert(MAMCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            MAMHailMax.insert(MAMCount,d)
        else: 
            MAMHailMax.insert(MAMCount,0)
    except KeyError: 
        MAMHailMax.insert(MAMCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            MAMWindMax.insert(MAMCount,d)
        else: 
            MAMWindMax.insert(MAMCount,0)
    except KeyError:
        MAMWindMax.insert(MAMCount,0)
    
    MAMReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        MAMReportDaysList.insert(MAMCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                MAMReportDays = MAMReportDays + 1
        MAMReportDaysList.insert(MAMCount, MAMReportDays)
    Date7 = Date7 + pd.DateOffset(years=1)
    Date8 = Date8 + pd.DateOffset(years=1)
    MAMCount = MAMCount + 1
    
#Create dataframes for each list
MAMdf1 = pd.DataFrame({'MAM Reports': MAMReports})
MAMdf2 = pd.DataFrame({'MAM Report Days': MAMReportDaysList})
MAMdf3 = pd.DataFrame({'MAM Fatalities': MAMDeath})
MAMdf4 = pd.DataFrame({'MAM Injuries': MAMInjury})
MAMdf5 = pd.DataFrame({'MAM Tornado Track': MAMTornadoTrack})
MAMdf6 = pd.DataFrame({'MAM Damages': MAMCost})
MAMdf7 = pd.DataFrame({'MAM Largest Hail': MAMHailMax})
MAMdf8 = pd.DataFrame({'MAM Strongest Wind': MAMWindMax})
MAMdf9 = pd.DataFrame({'MAM Widest Tornado': MAMTornadoWidth})

#concatenate the dataframes together
MAMdftot = pd.concat([
    MAMdf1, MAMdf2, MAMdf3, MAMdf4, 
    MAMdf5, MAMdf6, MAMdf7, MAMdf8, MAMdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
MAMdftot = MAMdftot.set_index([years])

In [6]:
#AMJ
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
AMJReports = []
AMJDeath = []
AMJInjury = []
AMJCost = []
AMJReportDaysList = []
AMJTornadoWidth = []
AMJTornadoTrack = []
AMJHailMax = []
AMJWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date9 = pd.to_datetime('04-01-1981')
Date10 = pd.to_datetime('06-30-1981')
AMJCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while AMJCount <= 29:
    x = df.loc[Date9:Date10]
    maxes = x.groupby(['EVENT_TYPE']).max()
    AMJReports.insert(AMJCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    AMJDeath.insert(AMJCount,y)
    z = x.INJURIES_DIRECT.sum()
    AMJInjury.insert(AMJCount,z)
    a = x.TOR_LENGTH.sum()
    AMJTornadoTrack.insert(AMJCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        AMJTornadoWidth.insert(AMJCount,b)
    else: 
        AMJTornadoWidth.insert(AMJCount,0)
    c = x.Total_Damages.sum()
    AMJCost.insert(AMJCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            AMJHailMax.insert(AMJCount,d)
        else: 
            AMJHailMax.insert(AMJCount,0)
    except KeyError: 
        AMJHailMax.insert(AMJCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            AMJWindMax.insert(AMJCount,d)
        else: 
            AMJWindMax.insert(AMJCount,0)
    except KeyError:
        AMJWindMax.insert(AMJCount,0)
    
    AMJReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        AMJReportDaysList.insert(AMJCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                AMJReportDays = AMJReportDays + 1
        AMJReportDaysList.insert(AMJCount, AMJReportDays)
    Date9 = Date9 + pd.DateOffset(years=1)
    Date10 = Date10 + pd.DateOffset(years=1)
    AMJCount = AMJCount + 1
    
#Create dataframes for each list
AMJdf1 = pd.DataFrame({'AMJ Reports': AMJReports})
AMJdf2 = pd.DataFrame({'AMJ Report Days': AMJReportDaysList})
AMJdf3 = pd.DataFrame({'AMJ Fatalities': AMJDeath})
AMJdf4 = pd.DataFrame({'AMJ Injuries': AMJInjury})
AMJdf5 = pd.DataFrame({'AMJ Tornado Track': AMJTornadoTrack})
AMJdf6 = pd.DataFrame({'AMJ Damages': AMJCost})
AMJdf7 = pd.DataFrame({'AMJ Largest Hail': AMJHailMax})
AMJdf8 = pd.DataFrame({'AMJ Strongest Wind': AMJWindMax})
AMJdf9 = pd.DataFrame({'AMJ Widest Tornado': AMJTornadoWidth})

#concatenate the dataframes together
AMJdftot = pd.concat([
    AMJdf1, AMJdf2, AMJdf3, AMJdf4,
    AMJdf5, AMJdf6, AMJdf7, AMJdf8, AMJdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
AMJdftot = AMJdftot.set_index([years])

In [15]:
#MJJ
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
MJJReports = []
MJJDeath = []
MJJInjury = []
MJJCost = []
MJJReportDaysList = []
MJJTornadoWidth = []
MJJTornadoTrack = []
MJJHailMax = []
MJJWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date11 = pd.to_datetime('05-01-1981')
Date12 = pd.to_datetime('07-31-1981')
MJJCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while MJJCount <= 29:
    x = df.loc[Date11:Date12]
    maxes = x.groupby(['EVENT_TYPE']).max()
    MJJReports.insert(MJJCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    MJJDeath.insert(MJJCount,y)
    z = x.INJURIES_DIRECT.sum()
    MJJInjury.insert(MJJCount,z)
    a = x.TOR_LENGTH.sum()
    MJJTornadoTrack.insert(MJJCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        MJJTornadoWidth.insert(MJJCount,b)
    else: 
        MJJTornadoWidth.insert(MJJCount,0)
    c = x.Total_Damages.sum()
    MJJCost.insert(MJJCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            MJJHailMax.insert(MJJCount,d)
        else: 
            MJJHailMax.insert(MJJCount,0)
    except KeyError: 
        MJJHailMax.insert(MJJCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            MJJWindMax.insert(MJJCount,d)
        else: 
            MJJWindMax.insert(MJJCount,0)
    except KeyError:
        MJJWindMax.insert(MJJCount,0)
    
    MJJReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        MJJReportDaysList.insert(MJJCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                MJJReportDays = MJJReportDays + 1
        MJJReportDaysList.insert(MJJCount, MJJReportDays)
    Date11 = Date11 + pd.DateOffset(years=1)
    Date12 = Date12 + pd.DateOffset(years=1)
    MJJCount = MJJCount + 1
    
#Create dataframes for each list
MJJdf1 = pd.DataFrame({'MJJ Reports': MJJReports})
MJJdf2 = pd.DataFrame({'MJJ Report Days': MJJReportDaysList})
MJJdf3 = pd.DataFrame({'MJJ Fatalities': MJJDeath})
MJJdf4 = pd.DataFrame({'MJJ Injuries': MJJInjury})
MJJdf5 = pd.DataFrame({'MJJ Tornado Track': MJJTornadoTrack})
MJJdf6 = pd.DataFrame({'MJJ Damages': MJJCost})
MJJdf7 = pd.DataFrame({'MJJ Largest Hail': MJJHailMax})
MJJdf8 = pd.DataFrame({'MJJ Strongest Wind': MJJWindMax})
MJJdf9 = pd.DataFrame({'MJJ Widest Tornado': MJJTornadoWidth})

#concatenate the dataframes together
MJJdftot = pd.concat([
    MJJdf1, MJJdf2, MJJdf3, MJJdf4,
    MJJdf5, MJJdf6, MJJdf7, MJJdf8, MJJdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
MJJdftot = MJJdftot.set_index([years])

In [8]:
#JJA
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
JJAReports = []
JJADeath = []
JJAInjury = []
JJACost = []
JJAReportDaysList = []
JJATornadoWidth = []
JJATornadoTrack = []
JJAHailMax = []
JJAWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date13 = pd.to_datetime('06-01-1981')
Date14 = pd.to_datetime('08-31-1981')
JJACount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while JJACount <= 29:
    x = df.loc[Date13:Date14]
    maxes = x.groupby(['EVENT_TYPE']).max()
    JJAReports.insert(JJACount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    JJADeath.insert(JJACount,y)
    z = x.INJURIES_DIRECT.sum()
    JJAInjury.insert(JJACount,z)
    a = x.TOR_LENGTH.sum()
    JJATornadoTrack.insert(JJACount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        JJATornadoWidth.insert(JJACount,b)
    else: 
        JJATornadoWidth.insert(JJACount,0)
    c = x.Total_Damages.sum()
    JJACost.insert(JJACount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            JJAHailMax.insert(JJACount,d)
        else: 
            JJAHailMax.insert(JJACount,0)
    except KeyError: 
        JJAHailMax.insert(JJACount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            JJAWindMax.insert(JJACount,d)
        else: 
            JJAWindMax.insert(JJACount,0)
    except KeyError:
        JJAWindMax.insert(JJACount,0)
    
    JJAReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        JJAReportDaysList.insert(JJACount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                JJAReportDays = JJAReportDays + 1
        JJAReportDaysList.insert(JJACount, JJAReportDays)
    Date13 = Date13 + pd.DateOffset(years=1)
    Date14 = Date14 + pd.DateOffset(years=1)
    JJACount = JJACount + 1
    
#Create dataframes for each list
JJAdf1 = pd.DataFrame({'JJA Reports': JJAReports})
JJAdf2 = pd.DataFrame({'JJA Report Days': JJAReportDaysList})
JJAdf3 = pd.DataFrame({'JJA Fatalities': JJADeath})
JJAdf4 = pd.DataFrame({'JJA Injuries': JJAInjury})
JJAdf5 = pd.DataFrame({'JJA Tornado Track': JJATornadoTrack})
JJAdf6 = pd.DataFrame({'JJA Damages': JJACost})
JJAdf7 = pd.DataFrame({'JJA Largest Hail': JJAHailMax})
JJAdf8 = pd.DataFrame({'JJA Strongest Wind': JJAWindMax})
JJAdf9 = pd.DataFrame({'JJA Widest Tornado': JJATornadoWidth})

#concatenate the dataframes together
JJAdftot = pd.concat([
    JJAdf1, JJAdf2, JJAdf3, JJAdf4,
    JJAdf5, JJAdf6, JJAdf7, JJAdf8, JJAdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
JJAdftot = JJAdftot.set_index([years])

In [9]:
#JAS
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
JASReports = []
JASDeath = []
JASInjury = []
JASCost = []
JASReportDaysList = []
JASTornadoWidth = []
JASTornadoTrack = []
JASHailMax = []
JASWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date15 = pd.to_datetime('07-01-1981')
Date16 = pd.to_datetime('09-30-1981')
JASCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while JASCount <= 29:
    x = df.loc[Date15:Date16]
    maxes = x.groupby(['EVENT_TYPE']).max()
    JASReports.insert(JASCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    JASDeath.insert(JASCount,y)
    z = x.INJURIES_DIRECT.sum()
    JASInjury.insert(JASCount,z)
    a = x.TOR_LENGTH.sum()
    JASTornadoTrack.insert(JASCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        JASTornadoWidth.insert(JASCount,b)
    else: 
        JASTornadoWidth.insert(JASCount,0)
    c = x.Total_Damages.sum()
    JASCost.insert(JASCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            JASHailMax.insert(JASCount,d)
        else: 
            JASHailMax.insert(JASCount,0)
    except KeyError: 
        JASHailMax.insert(JASCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            JASWindMax.insert(JASCount,d)
        else: 
            JASWindMax.insert(JASCount,0)
    except KeyError:
        JASWindMax.insert(JASCount,0)
    
    JASReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        JASReportDaysList.insert(JASCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                JASReportDays = JASReportDays + 1
        JASReportDaysList.insert(JASCount, JASReportDays)
    Date15 = Date15 + pd.DateOffset(years=1)
    Date16 = Date16 + pd.DateOffset(years=1)
    JASCount = JASCount + 1
    
#Create dataframes for each list
JASdf1 = pd.DataFrame({'JAS Reports': JASReports})
JASdf2 = pd.DataFrame({'JAS Report Days': JASReportDaysList})
JASdf3 = pd.DataFrame({'JAS Fatalities': JASDeath})
JASdf4 = pd.DataFrame({'JAS Injuries': JASInjury})
JASdf5 = pd.DataFrame({'JAS Tornado Track': JASTornadoTrack})
JASdf6 = pd.DataFrame({'JAS Damages': JASCost})
JASdf7 = pd.DataFrame({'JAS Largest Hail': JASHailMax})
JASdf8 = pd.DataFrame({'JAS Strongest Wind': JASWindMax})
JASdf9 = pd.DataFrame({'JAS Widest Tornado': JASTornadoWidth})

#concatenate the dataframes together
JASdftot = pd.concat([
    JASdf1, JASdf2, JASdf3, JASdf4, 
    JASdf5, JASdf6, JASdf7, JASdf8, JASdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
JASdftot = JASdftot.set_index([years])

In [10]:
#ASO
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
ASOReports = []
ASODeath = []
ASOInjury = []
ASOCost = []
ASOReportDaysList = []
ASOTornadoWidth = []
ASOTornadoTrack = []
ASOHailMax = []
ASOWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date17 = pd.to_datetime('08-01-1981')
Date18 = pd.to_datetime('10-31-1981')
ASOCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while ASOCount <= 29:
    x = df.loc[Date17:Date18]
    maxes = x.groupby(['EVENT_TYPE']).max()
    ASOReports.insert(ASOCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    ASODeath.insert(ASOCount,y)
    z = x.INJURIES_DIRECT.sum()
    ASOInjury.insert(ASOCount,z)
    a = x.TOR_LENGTH.sum()
    ASOTornadoTrack.insert(ASOCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        ASOTornadoWidth.insert(ASOCount,b)
    else: 
        ASOTornadoWidth.insert(ASOCount,0)
    c = x.Total_Damages.sum()
    ASOCost.insert(ASOCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            ASOHailMax.insert(ASOCount,d)
        else: 
            ASOHailMax.insert(ASOCount,0)
    except KeyError: 
        ASOHailMax.insert(ASOCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            ASOWindMax.insert(ASOCount,d)
        else: 
            ASOWindMax.insert(ASOCount,0)
    except KeyError:
        ASOWindMax.insert(ASOCount,0)
    
    ASOReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        ASOReportDaysList.insert(ASOCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                ASOReportDays = ASOReportDays + 1
        ASOReportDaysList.insert(ASOCount, ASOReportDays)
    Date17 = Date17 + pd.DateOffset(years=1)
    Date18 = Date18 + pd.DateOffset(years=1)
    ASOCount = ASOCount + 1
    
#Create dataframes for each list
ASOdf1 = pd.DataFrame({'ASO Reports': ASOReports})
ASOdf2 = pd.DataFrame({'ASO Report Days': ASOReportDaysList})
ASOdf3 = pd.DataFrame({'ASO Fatalities': ASODeath})
ASOdf4 = pd.DataFrame({'ASO Injuries': ASOInjury})
ASOdf5 = pd.DataFrame({'ASO Tornado Track': ASOTornadoTrack})
ASOdf6 = pd.DataFrame({'ASO Damages': ASOCost})
ASOdf7 = pd.DataFrame({'ASO Largest Hail': ASOHailMax})
ASOdf8 = pd.DataFrame({'ASO Strongest Wind': ASOWindMax})
ASOdf9 = pd.DataFrame({'ASO Widest Tornado': ASOTornadoWidth})

#concatenate the dataframes together
ASOdftot = pd.concat([
    ASOdf1, ASOdf2, ASOdf3, ASOdf4,
    ASOdf5, ASOdf6, ASOdf7, ASOdf8, ASOdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
ASOdftot = ASOdftot.set_index([years])

In [11]:
#SON
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
SONReports = []
SONDeath = []
SONInjury = []
SONCost = []
SONReportDaysList = []
SONTornadoWidth = []
SONTornadoTrack = []
SONHailMax = []
SONWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date19 = pd.to_datetime('09-01-1981')
Date20 = pd.to_datetime('11-30-1981')
SONCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while SONCount <= 29:
    x = df.loc[Date19:Date20]
    maxes = x.groupby(['EVENT_TYPE']).max()
    SONReports.insert(SONCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    SONDeath.insert(SONCount,y)
    z = x.INJURIES_DIRECT.sum()
    SONInjury.insert(SONCount,z)
    a = x.TOR_LENGTH.sum()
    SONTornadoTrack.insert(SONCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        SONTornadoWidth.insert(SONCount,b)
    else: 
        SONTornadoWidth.insert(SONCount,0)
    c = x.Total_Damages.sum()
    SONCost.insert(SONCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            SONHailMax.insert(SONCount,d)
        else: 
            SONHailMax.insert(SONCount,0)
    except KeyError: 
        SONHailMax.insert(SONCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            SONWindMax.insert(SONCount,d)
        else: 
            SONWindMax.insert(SONCount,0)
    except KeyError:
        SONWindMax.insert(SONCount,0)
    
    SONReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        SONReportDaysList.insert(SONCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                SONReportDays = SONReportDays + 1
        SONReportDaysList.insert(SONCount, SONReportDays)
    Date19 = Date19 + pd.DateOffset(years=1)
    Date20 = Date20 + pd.DateOffset(years=1)
    SONCount = SONCount + 1
    
#Create dataframes for each list
SONdf1 = pd.DataFrame({'SON Reports': SONReports})
SONdf2 = pd.DataFrame({'SON Report Days': SONReportDaysList})
SONdf3 = pd.DataFrame({'SON Fatalities': SONDeath})
SONdf4 = pd.DataFrame({'SON Injuries': SONInjury})
SONdf5 = pd.DataFrame({'SON Tornado Track': SONTornadoTrack})
SONdf6 = pd.DataFrame({'SON Damages': SONCost})
SONdf7 = pd.DataFrame({'SON Largest Hail': SONHailMax})
SONdf8 = pd.DataFrame({'SON Strongest Wind': SONWindMax})
SONdf9 = pd.DataFrame({'SON Widest Tornado': SONTornadoWidth})

#concatenate the dataframes together
SONdftot = pd.concat([
    SONdf1, SONdf2, SONdf3, SONdf4,
    SONdf5, SONdf6, SONdf7, SONdf8, SONdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
SONdftot = SONdftot.set_index([years])

In [12]:
#OND
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
ONDReports = []
ONDDeath = []
ONDInjury = []
ONDCost = []
ONDReportDaysList = []
ONDTornadoWidth = []
ONDTornadoTrack = []
ONDHailMax = []
ONDWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date21 = pd.to_datetime('10-01-1981')
Date22 = pd.to_datetime('12-31-1981')
ONDCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while ONDCount <= 29:
    x = df.loc[Date21:Date22]
    maxes = x.groupby(['EVENT_TYPE']).max()
    ONDReports.insert(ONDCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    ONDDeath.insert(ONDCount,y)
    z = x.INJURIES_DIRECT.sum()
    ONDInjury.insert(ONDCount,z)
    a = x.TOR_LENGTH.sum()
    ONDTornadoTrack.insert(ONDCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        ONDTornadoWidth.insert(ONDCount,b)
    else: 
        ONDTornadoWidth.insert(ONDCount,0)
    c = x.Total_Damages.sum()
    ONDCost.insert(ONDCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            ONDHailMax.insert(ONDCount,d)
        else: 
            ONDHailMax.insert(ONDCount,0)
    except KeyError: 
        ONDHailMax.insert(ONDCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            ONDWindMax.insert(ONDCount,d)
        else: 
            ONDWindMax.insert(ONDCount,0)
    except KeyError:
        ONDWindMax.insert(ONDCount,0)
    
    ONDReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        ONDReportDaysList.insert(ONDCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                ONDReportDays = ONDReportDays + 1
        ONDReportDaysList.insert(ONDCount, ONDReportDays)
    Date21 = Date21 + pd.DateOffset(years=1)
    Date22 = Date22 + pd.DateOffset(years=1)
    ONDCount = ONDCount + 1
    
#Create dataframes for each list
ONDdf1 = pd.DataFrame({'OND Reports': ONDReports})
ONDdf2 = pd.DataFrame({'OND Report Days': ONDReportDaysList})
ONDdf3 = pd.DataFrame({'OND Fatalities': ONDDeath})
ONDdf4 = pd.DataFrame({'OND Injuries': ONDInjury})
ONDdf5 = pd.DataFrame({'OND Tornado Track': ONDTornadoTrack})
ONDdf6 = pd.DataFrame({'OND Damages': ONDCost})
ONDdf7 = pd.DataFrame({'OND Largest Hail': ONDHailMax})
ONDdf8 = pd.DataFrame({'OND Strongest Wind': ONDWindMax})
ONDdf9 = pd.DataFrame({'OND Widest Tornado': ONDTornadoWidth})

#concatenate the dataframes together
ONDdftot = pd.concat([
    ONDdf1, ONDdf2, ONDdf3, ONDdf4,
    ONDdf5, ONDdf6, ONDdf7, ONDdf8, ONDdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
ONDdftot = ONDdftot.set_index([years])

In [13]:
#NDJ
Table = pd.read_csv('C:\Users\Ty Dickinson\Downloads\SevereStormsUpdated.csv', low_memory=False)
Table['BEGIN_DATE'] = pd.to_datetime(Table.BEGIN_DATE)

#adding new columns based on adjustments from existing columns
Table['Total_Damages'] = Table['DAMAGE_PROPERTY_NUM'] + Table['DAMAGE_CROPS_NUM']
Table['Z_Time'] = Table['BEGIN_TIME']
Table['Z_Day'] = Table['BEGIN_DATE']

pd.options.mode.chained_assignment = None

#ensuring that a blank cell is read as a 0 for later addition.
for i in range(0, len(Table.TOR_LENGTH)):
    if Table.TOR_LENGTH[i] == ' ':
        Table.TOR_LENGTH[i] = 0.0
for i in range(0, len(Table.TOR_WIDTH)):
    if Table.TOR_WIDTH[i] == ' ':
        Table.TOR_WIDTH[i] = 0.0
for i in range(0, len(Table.MAGNITUDE)):
    if Table.MAGNITUDE[i] == ' ':
        Table.MAGNITUDE[i] = 0.0
         
Table.TOR_LENGTH = Table.TOR_LENGTH.astype(float)  
Table.TOR_WIDTH = Table.TOR_WIDTH.astype(float)
Table.DEATHS_DIRECT = Table.DEATHS_DIRECT.astype(int)
Table.INJURIES_DIRECT = Table.INJURIES_DIRECT.astype(int)
Table.MAGNITUDE = Table.MAGNITUDE.astype(float)
Table.BEGIN_TIME = Table.BEGIN_TIME.astype(int)

#converting CST to Z 
for i in range(0, len(Table.BEGIN_TIME)):
    if Table.BEGIN_TIME[i] <= 1759:
        Table.Z_Time[i] = Table.BEGIN_TIME[i] + 600
        Table.Z_Day[i] = Table.BEGIN_DATE[i]
    else:
        Table.Z_Time[i] = (Table.BEGIN_TIME[i] + 600) - 2400
        Table.Z_Day[i] = Table.BEGIN_DATE[i] + pd.DateOffset(days=1)

#Defining lists to use for each indicator
NDJReports = []
NDJDeath = []
NDJInjury = []
NDJCost = []
NDJReportDaysList = []
NDJTornadoWidth = []
NDJTornadoTrack = []
NDJHailMax = []
NDJWindMax = []


#setting the index as date to use the df.loc expression 
df = Table.set_index(['BEGIN_DATE'])

#defining date ranges and converting them to a Datetime data type
Date23 = pd.to_datetime('11-01-1981')
Date24 = pd.to_datetime('01-31-1982')
NDJCount = 0

#while statements to cycle through seasons and calculate reports, report days, fatalities, injuries
#damages, maximum hail size, maximum wind magnitude, maximum tornado width, and total tornado path length
while NDJCount <= 29:
    x = df.loc[Date23:Date24]
    maxes = x.groupby(['EVENT_TYPE']).max()
    NDJReports.insert(NDJCount,len(x.index))
    y = x.DEATHS_DIRECT.sum()
    NDJDeath.insert(NDJCount,y)
    z = x.INJURIES_DIRECT.sum()
    NDJInjury.insert(NDJCount,z)
    a = x.TOR_LENGTH.sum()
    NDJTornadoTrack.insert(NDJCount,a)
    if not len(x.index) == 0 :
        b = max(x.TOR_WIDTH)
        NDJTornadoWidth.insert(NDJCount,b)
    else: 
        NDJTornadoWidth.insert(NDJCount,0)
    c = x.Total_Damages.sum()
    NDJCost.insert(NDJCount,c)
    
    #These try statements are for finding the maximum magnitude for wind and hail. 
    #If there are no wind/hail reports, a KeyError is raised and then its value in the list is 0
    #If there are no reports for the season at all, the values are set at 0 in the else block
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Hail']
            NDJHailMax.insert(NDJCount,d)
        else: 
            NDJHailMax.insert(NDJCount,0)
    except KeyError: 
        NDJHailMax.insert(NDJCount,0)
        
    try:
        if not len(x.index) == 0:
            d = maxes.MAGNITUDE['Thunderstorm Wind']
            NDJWindMax.insert(NDJCount,d)
        else: 
            NDJWindMax.insert(NDJCount,0)
    except KeyError:
        NDJWindMax.insert(NDJCount,0)
    
    NDJReportDays = 1
    #nested if statements to calculate report days based on 12Z to 12Z the next day as 1 day
    if len(x.index) == 0:
        NDJReportDaysList.insert(NDJCount, 0)
    else: 
        for i in range(1, len(x.index)):
            if ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] < 1200))):
                continue
            elif ((x.Z_Day[i] == x.Z_Day[i-1]) and ((x.Z_Time[i] >= 1200) and (x.Z_Time[i-1] >= 1200))):
                continue
            elif (x.Z_Day[i] == (x.Z_Day[i-1] + pd.DateOffset(days=1))) and ((x.Z_Time[i] < 1200) and (x.Z_Time[i-1] >= 1200)):
                continue
            else:
                NDJReportDays = NDJReportDays + 1
        NDJReportDaysList.insert(NDJCount, NDJReportDays)
    Date23 = Date23 + pd.DateOffset(years=1)
    Date24 = Date24 + pd.DateOffset(years=1)
    NDJCount = NDJCount + 1
    
#Create dataframes for each list
NDJdf1 = pd.DataFrame({'NDJ Reports': NDJReports})
NDJdf2 = pd.DataFrame({'NDJ Report Days': NDJReportDaysList})
NDJdf3 = pd.DataFrame({'NDJ Fatalities': NDJDeath})
NDJdf4 = pd.DataFrame({'NDJ Injuries': NDJInjury})
NDJdf5 = pd.DataFrame({'NDJ Tornado Track': NDJTornadoTrack})
NDJdf6 = pd.DataFrame({'NDJ Damages': NDJCost})
NDJdf7 = pd.DataFrame({'NDJ Largest Hail': NDJHailMax})
NDJdf8 = pd.DataFrame({'NDJ Strongest Wind': NDJWindMax})
NDJdf9 = pd.DataFrame({'NDJ Widest Tornado': NDJTornadoWidth})

#concatenate the dataframes together
NDJdftot = pd.concat([
    NDJdf1, NDJdf2, NDJdf3, NDJdf4,
    NDJdf5, NDJdf6, NDJdf7, NDJdf8, NDJdf9
], axis=1)
years = []
for i in range(1981,2011):
    years.insert(i,i)
NDJdftot = NDJdftot.set_index([years])

In [16]:
#DJFdftot.to_csv('DJFSevereImpacts.csv')
#JFMdftot.to_csv('JFMSevereImpacts.csv')
#FMAdftot.to_csv('FMASevereImpacts.csv')
#MAMdftot.to_csv('MAMSevereImpacts.csv')
#AMJdftot.to_csv('AMJSevereImpacts.csv')
MJJdftot.to_csv('MJJSevereImpacts.csv')
#JJAdftot.to_csv('JJASevereImpacts.csv')
#JASdftot.to_csv('JASSevereImpacts.csv')
#ASOdftot.to_csv('ASOSevereImpacts.csv')
#SONdftot.to_csv('SONSevereImpacts.csv')
#ONDdftot.to_csv('ONDSevereImpacts.csv')
#NDJdftot.to_csv('NDJSevereImpacts.csv')